In [1]:
import os 
import json 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")

In [49]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.nbcnews.com/news/world/worlds-workers-rally-may-day-france-braces-protests-rcna82204'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
article_content = soup.find('div', {'class': 'article-body'})

# Check if article_content is not None before proceeding
if article_content:
    # Extract the text content from the article_content variable
    text_content = article_content.get_text()

    # Save the extracted text to a file
    with open('article_content.txt', 'w', encoding='utf-8') as file:
        file.write(text_content)
else:
    print("Article content not found.")


In [53]:
def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

# Usage example:
file_path = '/home/cloudsuperadmin/news-sum/article_content.txt'
loaded_content = load_text_file(file_path)
print(loaded_content)


Link copiedMay 1, 2023, 12:03 PM UTC / Updated May 1, 2023, 1:31 PM UTC / Source: Associated PressBy Associated PressPeople squeezed by inflation and demanding economic justice took to the streets of cities across Asia and Europe to mark May Day on Monday, in a global outpouring of worker discontent not seen since before the Covid-19 pandemic sent the world into lockdowns.French unions pushed the president to scrap a higher retirement age. South Koreans pleaded for higher wages. Spanish lawyers demanded the right to take days off. Migrant domestic workers in Lebanon marched in a country plunged in economic crisis.While May Day is marked around the world on May 1 as a celebration of labor rights, this year’s rallies tapped into broader frustrations. Climate activists spraypainted a Louis Vuitton museum in Paris, and protesters in Germany demonstrated against violence targeting women and LGBTQ+ people.Celebrations were forced indoors in Pakistan and tinged with political tensions in Turk

In [58]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(loaded_content)

In [54]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 20,
    length_function = len,
)

texts = text_splitter.create_documents(loaded_content)

In [60]:
print(texts[0])

Link copiedMay 1, 2023, 12:03 PM UTC / Updated May 1, 2023, 1:31 PM UTC / Source: Associated PressBy Associated PressPeople squeezed by inflation and demanding economic justice took to the streets of cities across Asia and Europe to mark May Day on Monday, in a global outpouring of worker discontent not seen since before the Covid-19 pandemic sent the world into lockdowns.French unions pushed the president to scrap a higher retirement age. South Koreans pleaded for higher wages. Spanish lawyers demanded the right to take days off. Migrant domestic workers in Lebanon marched in a country plunged in economic crisis.While May Day is marked around the world on May 1 as a celebration of labor rights, this year’s rallies tapped into broader frustrations. Climate activists spraypainted a Louis Vuitton museum in Paris, and protesters in Germany demonstrated against violence targeting women and LGBTQ+ people.Celebrations were forced indoors in Pakistan and tinged with political tensions in Turk

In [62]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI
from langchain.chains import GraphQAChain



In [64]:
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

In [65]:
with open("/home/cloudsuperadmin/news-sum/news_content.txt") as f:
    all_text = f.read()

In [68]:
graph = index_creator.from_text(all_text)

In [69]:
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

In [70]:
chain.run("What the protesters in Germany demonstrated against?")



> Entering new GraphQAChain chain...
Entities Extracted:
 Germany
Full Context:
Germany demonstrated against violence targeting women and LGBTQ+ people

> Finished chain.


' The protesters in Germany demonstrated against violence targeting women and LGBTQ+ people.'

In [71]:
import re
from dateutil.parser import parse

def find_and_reformat_dates(all_text):
    # Regular expression pattern to find dates
    date_pattern = r'\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{1,2},\s+\d{4}\b'

    # Find all date occurrences in the text
    dates_found = re.findall(date_pattern, all_text)

    # Reformat each date found
    reformatted_dates = [parse(date).isoformat() for date in dates_found]

    # Replace dates in text with reformatted dates
    for original_date, reformatted_date in zip(dates_found, reformatted_dates):
        text = all_text.replace(original_date, reformatted_date)

    return text

In [72]:
reformatted_text = find_and_reformat_dates(all_text)
print(reformatted_text)

People squeezed by inflation and demanding economic justice took to the streets of cities across Asia and Europe to mark May Day on Monday, in a global outpouring of worker discontent not seen since before the Covid-19 pandemic sent the world into lockdowns.French unions pushed the president to scrap a higher retirement age. South Koreans pleaded for higher wages. Spanish lawyers demanded the right to take days off. Migrant domestic workers in Lebanon marched in a country plunged in economic crisis.While May Day is marked around the world on May 1 as a celebration of labor rights, this year’s rallies tapped into broader frustrations. Climate activists spraypainted a Louis Vuitton museum in Paris, and protesters in Germany demonstrated against violence targeting women and LGBTQ+ people.Celebrations were forced indoors in Pakistan and tinged with political tensions in Turkey, as both countries face high-stakes elections. Russia’s war in Ukraine overshadowed scaled-back events in Moscow, 

In [74]:
import re
from dateutil.parser import parse

# Function to extract all dates in ISO format from a text
def extract_iso_dates(all_text):
    # Regular expression pattern to find ISO dates
    iso_date_pattern = r'\d{4}-\d{2}-\d{2}'

    # Find all ISO date occurrences in the text
    iso_dates_found = re.findall(iso_date_pattern, all_text)

    return iso_dates_found


In [76]:
iso_dates = extract_iso_dates(reformatted_text)

for iso_date in iso_dates:
    print(iso_date)


2023-05-01
2023-05-01
2023-05-01


In [78]:
from langchain.indexes import GraphIndexCreator
from langchain.llms import OpenAI
from langchain.chains import GraphQAChain

# Initialize the GraphIndexCreator with OpenAI as the language model
index_creator = GraphIndexCreator(llm=OpenAI(temperature=0))

# Example news article text
data = reformatted_text

# Extract the knowledge graph from the text
graph = index_creator.from_text(data)

# Print the extracted triples (entities and relations)
triples = graph.get_triples()
print(triples)


[('France', 'a higher retirement age', 'pushed the president to scrap'), ('France', 'to raise the retirement age', "mobilized against President Emmanuel Macron's recent move"), ('France', 'Laurent Berger', "shows workers' hope to be treated differently"), ('France', 'Paris', 'police detained 22 people'), ('France', 'Lyon', 'police dispersed protesters'), ('South Korea', 'higher wages', 'pleaded for'), ('Spain', 'days off', 'demanded the right to take'), ('Lebanon', 'a country plunged in economic crisis', 'marched in'), ('Paris', 'a Louis Vuitton museum', 'spraypainted'), ('Germany', 'violence targeting women and LGBTQ+ people', 'demonstrated against'), ('Pakistan', 'May Day events', 'forced indoors'), ('Turkey', 'May Day events', 'tinged with political tensions'), ('Turkey', "reaching Istanbul's main square", 'police prevented a group of demonstrators'), ('Turkey', 'pushed back or detained', 'journalists trying to film demonstrators'), ('Russia', 'May Day celebrations', 'overshadowed s

In [79]:
# Initialize the GraphQAChain with the knowledge graph
chain = GraphQAChain.from_llm(OpenAI(temperature=0), graph=graph, verbose=True)

# Query the knowledge graph
answer = chain.run("What happened in Paris, France?")
#print(answer)



> Entering new GraphQAChain chain...
Entities Extracted:
 Paris, France
Full Context:
Paris spraypainted a Louis Vuitton museumFrance pushed the president to scrap a higher retirement age
France mobilized against President Emmanuel Macron's recent move to raise the retirement age
France shows workers' hope to be treated differently Laurent Berger
France police detained 22 people Paris
France police dispersed protesters Lyon

> Finished chain.
